# ConvLSTM for Patches
Trains a convolutional LSTM. The model input is a stack of 64x64 patches of the total area of interest, and the model output is a 64x64 patch with the predicted land type at the next time step. Requires about 12 GB RAM to run.

In [ ]:
%matplotlib inline
import keras_core as keras
import tensorflow as tf
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
SEED = 42
rng = np.random.default_rng(SEED)

Define parameters for patch size.

In [ ]:
PATCH_SIZE = 64  # Size of each patch in pixels
OVERLAP_SIZE = 32  # Number of pixels to advance before accessing the next patch
MAX_EMPTY_RATIO = 0.4  # Maximum percent of pixels in the image that can be zero
MIN_CHANGE_RATIO = 0.2  # Minimum percent of pixels that must change from the earliest to latest timestamp in the dataset
TIME_STEPS = 5  # Number of time steps

In [ ]:
images = []
files = sorted(glob('data/CONUS20*_ClipAOI*.tif'))
for f in files:
    with rasterio.open(f) as ds:
        data = ds.read(1)
        images.append(data)
images = np.array(images)
n_times = images.shape[0]
images.shape

In [ ]:
with rasterio.open('data/slope.tif') as ds:
    slopes = ds.read(1)
slopes[slopes <= 0] = 0
slopes = np.expand_dims(slopes, axis=-1)
slopes.shape

Compute 2D prefix sum arrays for the entire large image. When passing patches to the model during training, we want to exclude patches where the entire image or the majority of pixels are out of bounds (zero), and also images where the terrain is almost completely unchanged. Calculating the prefix sum arrays for the entire large image will allow fast querying of the number of zero pixels in any given patch.

In [ ]:
image = images[0, :, :]
zero_prefix = np.zeros_like(image, dtype=np.uint32)
zero_prefix[image == 0] = 1
zero_prefix = np.cumsum(np.cumsum(zero_prefix, axis=0, dtype=np.uint32), axis=1, dtype=np.uint32)
zero_prefix.shape

In [ ]:
def get_zero_pixels(i, j):
    """Calculates the number of zero pixels in the patch with corner at (i, j)."""
    zeros = zero_prefix[i + PATCH_SIZE - 1, j + PATCH_SIZE - 1]
    if i > 0 and j > 0:
        zeros += zero_prefix[i - 1, j - 1]
    if i > 0:
        zeros -= zero_prefix[i - 1, j + PATCH_SIZE - 1]
    if j > 0:
        zeros -= zero_prefix[i + PATCH_SIZE - 1, j - 1]
    return zeros

In [ ]:
change_prefix = np.zeros_like(image, dtype=np.uint32)
change_prefix[images[0, :, :] != images[-1, :, :]] = 1
change_prefix = np.cumsum(np.cumsum(change_prefix, axis=0, dtype=np.uint32), axis=1, dtype=np.uint32)
change_prefix.shape

In [ ]:
def get_changed_pixels(i, j):
    """Calculates the number of changed pixels in the patch with corner at (i, j)."""
    changes = change_prefix[i + PATCH_SIZE - 1, j + PATCH_SIZE - 1]
    if i > 0 and j > 0:
        changes += change_prefix[i - 1, j - 1]
    if i > 0:
        changes -= change_prefix[i - 1, j + PATCH_SIZE - 1]
    if j > 0:
        changes -= change_prefix[i + PATCH_SIZE - 1, j - 1]
    return changes

Determine the possible categories and normalize them to integer values starting at 0.

In [ ]:
categories, counts = np.unique(image, return_counts=True)
np.save('categories.npy', categories)
n_categories = categories.shape[0]
category_map = {categories[i]: i for i in range(n_categories)}
percents = counts / image.size * 100
del image
plt.bar(list(map(str, categories)), percents)
plt.title('Land Type Distribution')
plt.ylabel('Percent')
plt.show()
category_map

Using the prefix sum arrays, find the indices of every patch in the dataset that lies in the area of interest and has enough changed pixels

In [ ]:
indices = []
for i in range(0, images.shape[1] - PATCH_SIZE, OVERLAP_SIZE):
    for j in range(0, images.shape[2] - PATCH_SIZE, OVERLAP_SIZE):
        zeros = get_zero_pixels(i, j)
        if zeros >= PATCH_SIZE * PATCH_SIZE * MAX_EMPTY_RATIO:
            continue
        changes = get_changed_pixels(i, j)
        if changes < PATCH_SIZE * PATCH_SIZE * MIN_CHANGE_RATIO:
            continue
        indices.append((i, j))
del zero_prefix, change_prefix
indices = np.array(indices)
rng.shuffle(indices)
indices.shape

## Training the Model
Define training parameters.

In [ ]:
BATCH_SIZE = 64
EPOCHS = 1
VAL_SPLIT = 0.2

In [ ]:
val_size = int(indices.shape[0] * VAL_SPLIT)
val_indices = indices[:val_size, :]
train_indices = indices[val_size:, :]
train_indices.shape, val_indices.shape

Build the dataset using the list of patch indices. The full dataset is generated in-place using a generator function because it would be too large to fit in memory.

In [ ]:
def one_hot(x):
    encoded = np.zeros(x.shape + (n_categories,), dtype=np.uint8)
    for category, index in category_map.items():
        category_mask = (x == category)
        encoded[category_mask, index] = 1
    return encoded

def get_data(indices):
    sparse_encoder = np.vectorize(lambda x: category_map[x], otypes=[np.uint8])
    for i, j in indices:
        for k in range(n_times - TIME_STEPS):
            land = one_hot(images[k:k + TIME_STEPS, i:i + PATCH_SIZE, j:j + PATCH_SIZE])
            slope = slopes[i:i + PATCH_SIZE, j:j + PATCH_SIZE, :]
            x = {'land': land, 'slope': slope}
            y = sparse_encoder(images[k + TIME_STEPS, i:i + PATCH_SIZE, j:j + PATCH_SIZE])
            yield x, y

In [ ]:
np.save('val-indices.npy', val_indices)

In [ ]:
ds_sig = (
    {
        'land': tf.TensorSpec(shape=(TIME_STEPS, PATCH_SIZE, PATCH_SIZE, n_categories), dtype=tf.float32),
        'slope': tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, 1), dtype=tf.float32)
    },
    tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE), dtype=tf.uint8)
)

In [ ]:
train_ds = tf.data.Dataset.from_generator(lambda: get_data(train_indices), output_signature=ds_sig)
# Tell Keras the full size of the dataset so we get ETA in the progress bar
train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(train_indices.shape[0] * (n_times - TIME_STEPS)))
train_ds = train_ds.batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_generator(lambda: get_data(val_indices), output_signature=ds_sig)
val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(val_indices.shape[0] * (n_times - TIME_STEPS)))
val_ds = val_ds.batch(BATCH_SIZE)

In [ ]:
land_input = keras.layers.Input(shape=(TIME_STEPS, PATCH_SIZE, PATCH_SIZE, n_categories), name='land')
slope_input = keras.layers.Input(shape=(PATCH_SIZE, PATCH_SIZE, 1), name='slope')
x1 = keras.layers.ConvLSTM2D(64, kernel_size=(3, 3), padding='same', return_sequences=True, activation='relu')(land_input)
x1, = keras.layers.BatchNormalization()(x1),
x1 = keras.layers.ConvLSTM2D(64, kernel_size=(3, 3), padding='same', activation='relu')(x1)
x2 = keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(slope_input)
x = keras.layers.Concatenate(axis=-1)([x1, x2])
output = keras.layers.Conv2D(n_categories, kernel_size=(3, 3), padding='same', activation='softmax')(x)
model = keras.Model(inputs=(land_input, slope_input), outputs=output)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name='acc')
    ]
)
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [ ]:
model.fit(train_ds, epochs=EPOCHS)

In [ ]:
model.save('lttn-convlstm-patches-slope.keras')

## Evaluating the model
Load the saved model (optional if run from the same session used to train above)

In [ ]:
model = keras.saving.load_model('lttn-convlstm-patches-slope.keras')
model.summary()

In [ ]:
categories = np.load('categories.npy')
n_categories = categories.shape[0]
category_map = {categories[i]: i for i in range(n_categories)}
category_map

Run the model on the validation set

In [ ]:
model.evaluate(val_ds, return_dict=True)

In [ ]:
val_indices = np.load('val-indices.npy')
val_indices.shape

In [ ]:
val_ds = tf.data.Dataset.from_generator(lambda: get_data(val_indices), output_signature=ds_sig)
val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(val_indices.shape[0] * (n_times - TIME_STEPS)))
val_ds = val_ds.batch(BATCH_SIZE)
len(val_ds)

In [ ]:
val_pred = model.predict(val_ds)
val_pred.shape

In [ ]:
val_y = np.concatenate([y for _, y in val_ds], axis=0)
val_y.shape

## Making predictions
Run the first 2 cells from the evaluation section to load the model first. Some cells here also require other cells from the training section to be ran, to define to 

In [ ]:
INFER_BATCH_SIZE = 128

Run the model to predict the landscape of a random patch.

In [ ]:
x = 10192 # random.randint(2000, 16000)
y = 14735 # random.randint(3000, 32000)
patch = images[-TIME_STEPS:, x:x + PATCH_SIZE, y:y + PATCH_SIZE]
fig, axs = plt.subplots(1, 5, figsize=(10, 50))
for i, ax in enumerate(axs):
    ax.imshow(patch[i, :, :])
plt.show()

In [ ]:
patch = one_hot(patch)
patch = np.expand_dims(patch, axis=0)
result = model.predict(patch)
result = np.squeeze(result, axis=0)
result = np.argmax(result, axis=-1)
plt.imshow(result)
plt.show()

Set parameters for generating the large prediction map

In [ ]:
x = 10192 # random.randint(2000, 16000)
y = 14735 # random.randint(3000, 32000)
size = 16

In [ ]:
def patch_generator(step):
    for i in range(0, PATCH_SIZE * size - (PATCH_SIZE - step), step):
        for j in range(0, PATCH_SIZE * size - (PATCH_SIZE - step), step):
            patch = images[-TIME_STEPS:, x + i:x + i + PATCH_SIZE, y + j:y + j + PATCH_SIZE]
            patch = one_hot(patch)
            yield i, j, patch

def batch_patch_generator(gen):
    coords = []
    batch = []
    n = 0
    for i, j, patch in gen:
        coords.append((i, j))
        batch.append(patch)
        n += 1
        if n == INFER_BATCH_SIZE:
            yield np.array(coords), np.stack(batch)
            coords = []
            batch = []
            n = 0
    if n:
        yield np.array(coords), np.stack(batch)

Construct a large map of predictions by running the model on several consecutive patches without overlap.

In [ ]:
image = np.zeros((PATCH_SIZE * size, PATCH_SIZE * size), dtype=np.uint8)
progbar = keras.utils.Progbar(np.ceil(len(range(0, PATCH_SIZE * size, PATCH_SIZE)) ** 2 / INFER_BATCH_SIZE), unit_name='batch')
gen = batch_patch_generator(patch_generator(PATCH_SIZE))
n = 0
for coords, batch in gen:
    result = model.predict(batch, verbose=0)
    result = np.argmax(result, axis=-1)
    for i in range(len(coords)):
        image[coords[i, 0]:coords[i, 0] + PATCH_SIZE, coords[i, 1]:coords[i, 1] + PATCH_SIZE] = result[i]
    n += 1
    progbar.update(n)
plt.imshow(image)
plt.show()

Construct a large map of predictions with overlap. The probability distributions at each pixel are summed with contribution from all overlapping patches before `argmax` is called.

In [ ]:
probs = np.zeros((PATCH_SIZE * size, PATCH_SIZE * size, n_categories), dtype=np.float32)
progbar = keras.utils.Progbar(np.ceil(len(range(0, PATCH_SIZE * size - OVERLAP_SIZE, OVERLAP_SIZE)) ** 2 / INFER_BATCH_SIZE), unit_name='batch')
gen = batch_patch_generator(patch_generator(OVERLAP_SIZE))
n = 0
for coords, batch in gen:
    result = model.predict(batch, verbose=0)
    for i in range(len(coords)):
        probs[coords[i, 0]:coords[i, 0] + PATCH_SIZE, coords[i, 1]:coords[i, 1] + PATCH_SIZE] += result[i]
    n += 1
    progbar.update(n)
image = np.argmax(probs, axis=-1)
plt.imshow(image)
plt.show()

Build a map over the entire area of interest. Be sure to run the cell creating `zero_prefix` first.

In [ ]:
indices = []
for i in range(0, images.shape[1] - PATCH_SIZE, OVERLAP_SIZE):
    for j in range(0, images.shape[2] - PATCH_SIZE, OVERLAP_SIZE):
        zeros = get_zero_pixels(i, j)
        if zeros == PATCH_SIZE * PATCH_SIZE:
            continue  # Don't need to run any prediction if the slice is entirely blank
        indices.append((i, j))
del zero_prefix
indices = np.array(indices)
indices.shape

In [ ]:
def full_patch_generator():
    for i, j in indices:
        patch = images[-TIME_STEPS:, i:i + PATCH_SIZE, j:j + PATCH_SIZE]
        patch = one_hot(patch)
        yield i, j, patch

In [ ]:
probs = np.zeros(images.shape[1:] + (n_categories,), dtype=np.float32)
progbar = keras.utils.Progbar(np.ceil(len(indices) / INFER_BATCH_SIZE), unit_name='batch')
gen = batch_patch_generator(full_patch_generator())
n = 0
for coords, batch in gen:
    result = model.predict(batch, verbose=0)
    for i in range(len(coords)):
        probs[coords[i, 0]:coords[i, 0] + PATCH_SIZE, coords[i, 1]:coords[i, 1] + PATCH_SIZE] += result[i]
    n += 1
    progbar.update(n)
image = np.argmax(probs, axis=-1)
del probs
image.shape

In [ ]:
image = np.vectorize(lambda x: categories[x])(image)
image.shape

In [ ]:
np.save('full-map.npy', image)

In [ ]:
np.unique(image, return_counts=True)

Load the numpy array and save the data as a GeoTIFF file.

In [ ]:
image = np.load('full-map.npy')
image.shape

In [ ]:
with rasterio.open('data/CONUS2001_ClipAOI.tif') as src:
    profile = src.profile.copy()
    with rasterio.open('prediction-1.tif', 'w', **profile) as dst:
        dst.write(image, 1)